In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
save_dir = "./t5_base_subject_model"
tokenizer = T5Tokenizer.from_pretrained(save_dir)
model = T5ForConditionalGeneration.from_pretrained(save_dir)


import torch
device = torch.device("cpu")
model.to(device)
model.eval()


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
import pandas as pd

test_df = pd.read_csv('test.csv')  
val_df = pd.read_csv('val.csv')

In [8]:
from datasets import Dataset

val_dataset = Dataset.from_pandas(val_df[['input_text', 'target_text']].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df[['input_text', 'target_text']].reset_index(drop=True))

from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def tokenize(row):
    input_enc = tokenizer(
        row["input_text"],
        padding="max_length",
        truncation=True,
        max_length = 512
    )

    target_enc = tokenizer(
        row["target_text"],
        padding="max_length",
        truncation=True,
        max_length = 64
    )

    labels = [
        (token if token != tokenizer.pad_token_id else -100)
        for token in target_enc["input_ids"]
    ]

    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "labels": target_enc["input_ids"]
    }
    
    
tokenized_val = val_dataset.map(tokenize, batched=False)
tokenized_test = test_dataset.map(tokenize, batched=False)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/441 [00:00<?, ? examples/s]

Map:   0%|          | 0/441 [00:00<?, ? examples/s]

In [ ]:
from transformers import pipeline
from evaluate import load
import torch
# model.to("cpu")
model.to("cuda")



generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,             #
    batch_size=1          
)

rouge = load("rouge")

preds, refs = [], []

for i, text in enumerate(test_df["input_text"][:441]):
    try:
        out = generator(text, max_length=64, num_beams=2)
        preds.append(out[0]["generated_text"])

        label = tokenized_test["labels"][i]
        clean = [tok if tok != -100 else tokenizer.pad_token_id for tok in label]
        refs.append(tokenizer.decode(clean, skip_special_tokens=True))
    except Exception as e:
        print(f"Error at {i}: {e}")
        preds.append("FAILED")
        refs.append("")
        
    torch.cuda.empty_cache()

print("Done.")


metrics = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
print("VAL ROUGE:", {k: round(v,4) for k,v in metrics.items()})








Device set to use cuda:0


Error at 51: CUDA out of memory. Tried to allocate 62.86 GiB. GPU 0 has a total capacity of 23.99 GiB of which 20.00 GiB is free. Of the allocated memory 2.21 GiB is allocated by PyTorch, and 183.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error at 349: CUDA out of memory. Tried to allocate 12.43 GiB. GPU 0 has a total capacity of 23.99 GiB of which 0 bytes is free. Of the allocated memory 26.77 GiB is allocated by PyTorch, and 192.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Done.
VAL ROUGE: {'rouge1': 

ImportError: To be able to use evaluate-metric/bertscore, you need to install the following dependencies['bert_score'] using 'pip install bert_score' for instance'

In [25]:
bertscore = load("bertscore")
score = bertscore.compute(predictions=preds, references=refs, lang="en")
print("BERTScore F1:", round(sum(score["f1"])/len(score["f1"]), 4))

meteor = load("meteor")
score = meteor.compute(predictions=preds, references=refs)
print("METEOR:", round(score["meteor"], 4))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\DELL\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8703


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...


METEOR: 0.3365


In [22]:
print(len(test_df), len(preds))

df_out = pd.DataFrame({
    "input_text": test_df["input_text"].tolist(),
    "target_text": test_df["target_text"].tolist(),
    "prediction": preds
})
df_out.to_csv("t5_subject_predictions.csv", index=False)
print("Saved t5_subject_predictions.csv")

441 441
Saved t5_subject_predictions.csv


Device set to use cuda:0


Error at 409: CUDA out of memory. Tried to allocate 4.09 GiB. GPU 0 has a total capacity of 23.99 GiB of which 0 bytes is free. Of the allocated memory 34.75 GiB is allocated by PyTorch, and 191.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error at 415: CUDA out of memory. Tried to allocate 12.17 GiB. GPU 0 has a total capacity of 23.99 GiB of which 0 bytes is free. Of the allocated memory 26.23 GiB is allocated by PyTorch, and 195.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Done.
VAL ROUGE: {'rouge1': 0